# Assigment 3: Greedy Algorithms

In this assignment, we will explore greedy algorithms for makespan scheduling. We will see how a greedy algorithm can sometimes provide a solution that is guaranteed to be within some constant factor of the best possible solution. Please fill out the missing answers and the missing code below. Note that the coding part of this assignment should be simple given where we are in this class but the ungraded answers will hopefully be most instructive for this assignment.

See https://jeffe.cs.illinois.edu/teaching/algorithms/notes/J-approx.pdf section J1 for more details.
 
  
## Problem 1: Makespan Scheduling.

Let us consider $n$ jobs that take times $T_1, \ldots, T_n$ to complete where each $T_i > 0$. We have $m \geq 2$ processors to process these jobs. Our goal is to assign these jobs to the processor.

An assignment is modeled as an array $A: [A_1, \ldots, A_n]$ wherein each $A_i$ represents the number of the processor to which job $i$ is assigned. Eg., $A_3 = 4$ means that job number $3$ is assigned to processor $4$.
Therefore each $A_i \in \{ 1, \ldots, m \}$.

Once the assignment is complete, each processor runs the jobs assigned to it under some order. 

### Question 1

Let $M_j$ be the total time taken by some processor $j$ to complete all the jobs assigned to it.

Write down an expression for $M_j$? We will not grade your answer but you may be able to check against the provided solutions.

### MakeSpan of an Assignment (Def)
$$\newcommand\mspan{\sf MakeSpan }$$

The makespan of an assignment $A$ denoted $\mspan(A)$ is the maximum among the total times taken by each processor. Formally, 

$$\mspan(A) = \max_{j=1}^m M_j$$

The makespan of an assignment denotes the total time taken to complete all the jobs with the processors running in parallel since it measures the time taken by the processor which takes the longest to complete all its assigned tasks.

#### Example

Consider jobs with times $[T_1: 2,\ T_2: 2,\ T_3: 2,\ T_4: 2,\ T_5: 2,\ T_6: 2,\  T_7: 3]$ and $m = 3$ processors.

Consider the assignment $A: [A_1: 1,\ A_2: 1,\ A_3: 2,\ A_4: 2,\ A_5: 3,\ A_6: 3,\ A_7: 2 ]$.

### Question 2 

Write down the total times taken by each processor under the given assignment. What is the makespan of this assignment? Is there a better assignment of jobs to processor that can reduce the makespan? If so what is it?


## Problem A: Calculate Makespan of an Assignment

In [8]:
def compute_makespan(times, m, assign):
    # times is an array of job times of size n
    # m is the number of processors
    # assign is an array of size n whose entries are between 0 to m-1 
    # indicating the processor number for
    # the corresponding job.
    # Return: makespan of the assignment
    # your code here
    ms = [0] * m # Makespan array for processors
    for c, a in enumerate(assign):
        ms[a] = ms[a] + times[c]
        
    return max(ms)

In [11]:
## BEGIN TESTS
print('Test 1 ... ', end = '')
times = [2, 2, 2, 2, 3, 3, 2]
assigns = [0, 0, 0, 0, 1, 1, 2]
m = 3
s = compute_makespan(times, m, assigns)
assert s == 8, f'Expected makespan is 8, your code returned: {s}'
print(' passed!')

print('Test 2 ...', end='')
times = [2, 1, 2, 2, 1, 3, 2, 1, 1, 3]
assigns = [0, 1, 0, 1, 0, 1, 0, 1, 0, 1]
m = 3
s = compute_makespan(times, m, assigns)
assert s == 10, f' Expected makespan is 10, your code returned {s}'
print('  passed!')
print('Tests passed: 10 points!')

## END TESTS

Test 1 ...  passed!
Test 2 ...  passed!
Tests passed: 10 points!


### Minimizing Makespan

Given a list $T: [T_1, \ldots, T_n]$ of job times and $m \geq 2$ processors, we wish to find an assignment that minimizes the overall makespan.


### Question 3

What is the number of possible assignments for a problem with $n = 1000$ jobs on $m = 10$ processors?

As you will notice from the answer to the previous question, the number of possible assignments to a typical scheduling problem may well exceed the number of atoms in our Galaxy. Going through each and every one of them to find out the one that will minimize the makespan is impractical.  Furthermore, next module will study NP completeness. We will see that some problems including makespan scheduling are somehow inherently harder to solve on a computer. Thus, there are no known efficient solutions that solve for the optimal solution.

Therefore, we will (hat tip to the brilliant mathematician/computer scientist Ronald Graham https://en.wikipedia.org/wiki/Ronald_Graham) propose a simple greedy algorithm for makespan minimization.

## Greedy Makespan Minimization

The idea is simple: we go through each job and assign it to the processor that currently has the least load.

~~~
greedy_min_make_span(T, m):
  # T is an array of n numbers, m >= 2
  A = [Nil, ... , Nil] # Initialize the assignments to nil (array size n)
  M = [ 0, 0, ...., 0] # initialize the current load of each processor to 0 (array size m)
  for i = 1 to n
    find processor j for which M[j] is the least.
    A[i] = j
    M[j] = M[j] + T[i]
 # Assignment achieves a makespan of max(M[1], .. M[m])
 return A
~~~

### Question 4

What is the running time of the greedy makespan algorithm? What data structure would you use to achieve a running time of $n \log(m)$.

Finding the min element of array M is O(m), we also have to go through all jobs O(n)
Running time => O(n * m)

If we keep the list M as a min-heap, inserts would take O(logm), which would make the algorithm nlog(m) overall

## Problem B: Implement the Greedy Makespan Algorithm

In [16]:
def greedy_makespan_min(times, m):
    # times is a list of n jobs.
    assert len(times) >= 1
    assert all(elt >= 0 for elt in times)
    assert m >= 2
    n = len(times)
    # please do not reorder the jobs in times or else tests will fail.
    # you can implement a priority queue if you would like.
    # use https://docs.python.org/3/library/heapq.html heapq data structure 
    # Return a tuple of two things: 
    #    - Assignment list of n numbers from 0 to m-1
    #    - The makespan of your assignment
    # your code here
    A = [-1] * n # Using -1 instead of Nil
    M = [0] * m
    for i in range(n):
        j = M.index(min(M)) # Get the index/id of processor with minimum load
        A[i] = j
        M[j] = M[j] + times[i]
        
    ms = compute_makespan(times, m, A)
    return (A,ms)

In [17]:
## BEGIN TESTS
def do_test(times, m, expected):
    (a, makespan) = greedy_makespan_min(times,m )
    print('\t Assignment returned: ', a)
    print('\t Claimed makespan: ', makespan)
    assert compute_makespan(times, m, a) == makespan, 'Assignment returned is not consistent with the reported makespan'
    assert makespan == expected, f'Expected makespan should be {expected}, your core returned {makespan}'
    print('Passed')
print('Test 1:')
times = [2, 2, 2, 2, 2, 2, 2, 2, 3] 
m = 3
expected = 7
do_test(times, m, expected)

print('Test 2:')
times = [1]*20 + [5]
m = 5
expected =9
do_test(times, m, expected)

print('Test 3:')
times = [1]*40 + [2]
m = 20
expected = 4
do_test(times, m, expected)
print('All tests passed: 15 points!')
## END TESTS

Test 1:
	 Assignment returned:  [0, 1, 2, 0, 1, 2, 0, 1, 2]
	 Claimed makespan:  7
Passed
Test 2:
	 Assignment returned:  [0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0]
	 Claimed makespan:  9
Passed
Test 3:
	 Assignment returned:  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 0]
	 Claimed makespan:  4
Passed
All tests passed: 15 points!


## Question 5

Construct a set of timings for $n$ jobs with $m \geq 2 $ processors that shows that greedy solution can be strictly worse than the best solution.